In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#### 훈련용 디바이스 가져오기

In [2]:
# 사용 가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 훈련
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


#### 클래스 정의

In [3]:
# 파이토치에서 신경망 모듈 가져오기
import torch.nn as nn

# 파이토치에서 모든 신경망 모듈의 베이스 클래스인 nn.Module을 상속하는 NeuralNetwork 클래스를 정의합니다.
class NeuralNetwork(nn.Module):
    # 생성자 메서드로 NeuralNetwork 객체를 초기화합니다.
    def __init__(self):
        # 슈퍼클래스의 생성자 호출 (nn.Module)
        super().__init__()
        
        # 입력 텐서를 평탄화할 레이어 정의하기
        # 입력 텐서가 다차원 텐서(예: 이미지)인 경우, 이 레이어는 이를 1차원 텐서로 평탄화합니다.
        self.flatten = nn.Flatten()
        
        # 레이어의 순차적 컨테이너 정의하기
        # 레이어는 정의된 순서대로 적용됩니다.
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),  # First linear layer with input size 28*28 and output size 512
            nn.ReLU(),             # ReLU activation function applied element-wise
            nn.Linear(512, 512),   # Second linear layer with input size 512 and output size 512
            nn.ReLU(),             # ReLU activation function applied element-wise
            nn.Linear(512, 10),    # Third linear layer with input size 512 and output size 10
        )

    # 신경망의 포워드 전달을 정의하는 포워드 메서드
    def forward(self, x):
        # 입력 텐서 평탄화
        x = self.flatten(x)
        
        # 평탄화된 텐서에 순차적인 레이어 컨테이너 적용하기
        logits = self.linear_relu_stack(x)
        
        # 출력 텐서 반환
        return logits

In [4]:
# 신경망의 인스턴스를 생성하여 device를 지정하여 구조를 출력합니다.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
# 28x28 단일 이미지를 나타내는 크기 (1, 28, 28)의 랜덤 텐서를 생성합니다.
X = torch.rand(1, 28, 28, device=device)

# 입력 텐서를 신경망 모델에 전달하여 logits을 얻습니다.
# 'model' 오브젝트는 정의되어 로드된 신경망 클래스의 인스턴스여야 합니다.
logits = model(X)

# 소프트맥스 함수를 적용하여 logits을 확률로 변환합니다.
# 'dim=1' 인수는 1차원(열)을 따라 Softmax 함수를 적용합니다.
pred_probab = nn.Softmax(dim=1)(logits)

# 1차원을 따라 pred_probab에서 최대값의 인덱스 찾기
# y_pred에 할당된 인덱스는 예측된 클래스에 해당합니다.
y_pred = pred_probab.argmax(1)

# 예상 클래스 인쇄
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


### 모델 레이어
* FashionMNIST 모델의 레이어를 세분화 
* 28x28 크기의 이미지 3개로 구성된 샘플 미니배치를 가져옴

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


####  nn.Flatten
* 각 2D 28x28 이미지를 784 픽셀 값의 연속 배열로 변환하기 위해 nn.Flatten 레이어를 초기화

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


####  nn.Linear
* 저장된 가중치와 바이어스를 사용하여 입력에 선형 변환을 적용하는 모듈

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU
* 비선형 활성화 함수는 모델의 입력과 출력 사이에 복잡한 매핑을 생성하는 것
*  비선형 활성화함수는 선형 변환 후에 적용되어 비선형성을 도입함으로써 신경망이 다양한 현상을 학습할 수 있도록 도와줍니다.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.4743, -0.0690,  0.3589,  0.4472, -0.1861,  0.1954, -0.0496, -0.3752,
          0.3133,  0.1363,  0.0435,  0.1738, -0.1798,  0.2668, -0.4744,  0.5998,
          0.0663, -0.5331,  0.3837,  0.5801],
        [ 0.7383, -0.2533,  0.2000,  0.2397, -0.0026,  0.1583,  0.2527, -0.1919,
          0.4965, -0.2452, -0.3168,  0.1904, -0.3446,  0.2073, -0.1664,  0.4459,
         -0.1215, -0.2900,  0.3893,  0.4784],
        [ 0.6156, -0.3214, -0.1527,  0.3308, -0.2316,  0.2332, -0.0663, -0.2778,
          0.2999, -0.0057, -0.1131,  0.0179,  0.0189,  0.2025, -0.3720,  0.2651,
          0.0083, -0.2604,  0.4799,  0.4392]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4743, 0.0000, 0.3589, 0.4472, 0.0000, 0.1954, 0.0000, 0.0000, 0.3133,
         0.1363, 0.0435, 0.1738, 0.0000, 0.2668, 0.0000, 0.5998, 0.0663, 0.0000,
         0.3837, 0.5801],
        [0.7383, 0.0000, 0.2000, 0.2397, 0.0000, 0.1583, 0.2527, 0.0000, 0.4965,
         0.0000, 0.0000, 0.1904, 0.0000, 0.2073, 0.00

#### nn.시퀀셜
* nn.Sequential은 모듈의 정렬된 컨테이너
*  데이터는 정의된 것과 동일한 순서로 모든 모듈을 통과
* 시퀀셜 컨테이너를 사용하여 seq_modules와 같은 빠른 네트워크를 구성할 수 있습니다.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax
* 각 클래스에 대한 모델의 예측 확률을 나타내는 [0, 1] 값으로 스케일링
* dim 파라미터는 값이 1로 합산되어야 하는 차원을 나타냅니다.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#### 모델 파라미터
* 레이어는 훈련 중에 최적화된 가중치와 편향이 연관되어 있습니다.
* nn.Module을 서브클래싱하면 모델 오브젝트 내부에 정의된 모든 필드를 자동으로 추적
* 모델의 parameters() 또는 named_parameters() 메서드를 사용하여 모든 파라미터에 액세스할 수 있습니다.

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0046, -0.0135, -0.0105,  ..., -0.0337,  0.0036, -0.0215],
        [ 0.0048,  0.0033,  0.0043,  ..., -0.0307,  0.0083,  0.0193]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0106, -0.0055], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0257, -0.0071, -0.0006,  ...,  0.0047, -0.0424, -0.0393],
        [-0.0225, -0.0185,  0.0070,  ...,  0.0263, -0.0343, -0.0136]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

In [ ]:
self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), 
            nn.ReLU(),            
            nn.Linear(512, 512),   
            nn.ReLU(),             
            nn.Linear(512, 10),    
        )